In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/DL_Project

/content/gdrive/MyDrive/DL_Project


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17661675809291784419, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15505193728
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9277990931717346440
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"]

In [4]:
import numpy as np
# !pip install torch
import torch
import torch.optim
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from PIL import Image
import time
import csv
import os
from os import path

batch_size = 64
n_workers = 2

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        imgs = os.listdir(main_dir)
        self.total_imgs = []
        for i in imgs:
          if 'frame' in i:
            self.total_imgs.append(i) 

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image

transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),])

dset = CustomDataSet("unique-142p", transform)
loader = DataLoader(dset, batch_size=batch_size, shuffle=True, num_workers=n_workers, pin_memory=False, drop_last=True)

dset_0 = CustomDataSet("scene-change-examples/scene-0", transform)
loader_0 = DataLoader(dset, batch_size=batch_size, shuffle=False, num_workers=n_workers, pin_memory=False, drop_last=False)



In [ ]:
def reconstruction_loss(x, x_recon, distribution):
    batch_size = x.size(0)
    assert batch_size != 0

    if distribution == 'bernoulli':
        recon_loss = F.binary_cross_entropy_with_logits(x_recon, x, size_average=False).div(batch_size)
    elif distribution == 'gaussian':
        x_recon = F.sigmoid(x_recon)
        recon_loss = F.mse_loss(x_recon, x, size_average=False).div(batch_size)
    else:
        recon_loss = None

    return recon_loss


def kl_divergence(mu, logvar):
    batch_size = mu.size(0)
    assert batch_size != 0
    if mu.data.ndimension() == 4:
        mu = mu.view(mu.size(0), mu.size(1))
    if logvar.data.ndimension() == 4:
        logvar = logvar.view(logvar.size(0), logvar.size(1))

    klds = -0.5*(1 + logvar - mu.pow(2) - logvar.exp())
    total_kld = klds.sum(1).mean(0, True)
    dimension_wise_kld = klds.mean(0)
    mean_kld = klds.mean(1).mean(0, True)

    return total_kld, dimension_wise_kld, mean_kld

In [ ]:
def reparametrize(mu, logvar):
    std = logvar.div(2).exp()
    eps = Variable(std.data.new(std.size()).normal_())
    return mu + std*eps


class View(nn.Module):
    def __init__(self, size):
        super(View, self).__init__()
        self.size = size

    def forward(self, tensor):
        return tensor.view(self.size)

In [ ]:
class BetaVAE_H(nn.Module):
    """Model proposed in original beta-VAE paper(Higgins et al, ICLR, 2017)."""

    def __init__(self, z_dim=10, nc=3):
        super(BetaVAE_H, self).__init__()
        self.z_dim = z_dim
        self.nc = nc
        self.encoder = nn.Sequential(
            nn.Conv2d(nc, 32, 4, 2, 1),          # B,  32, 32, 32
            nn.ReLU(True),
            nn.Conv2d(32, 32, 4, 2, 1),          # B,  32, 16, 16
            nn.ReLU(True),
            nn.Conv2d(32, 64, 4, 2, 1),          # B,  64,  8,  8
            nn.ReLU(True),
            nn.Conv2d(64, 64, 4, 2, 1),          # B,  64,  4,  4
            nn.ReLU(True),
            nn.Conv2d(64, 256, 4, 1),            # B, 256,  1,  1
            nn.ReLU(True),
            View((-1, 256*1*1)),                 # B, 256
            nn.Linear(256, z_dim*2),             # B, z_dim*2
        )
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),               # B, 256
            View((-1, 256, 1, 1)),               # B, 256,  1,  1
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 64, 4),      # B,  64,  4,  4
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 64, 4, 2, 1), # B,  64,  8,  8
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), # B,  32, 16, 16
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, 4, 2, 1), # B,  32, 32, 32
            nn.ReLU(True),
            nn.ConvTranspose2d(32, nc, 4, 2, 1),  # B, nc, 64, 64
        )

        self.weight_init()

    def weight_init(self):
        for block in self._modules:
            for m in self._modules[block]:
                kaiming_init(m)

    def forward(self, x, train=True):
        distributions = self.encoder(x)
        mu = distributions[:, :self.z_dim]
        logvar = distributions[:, self.z_dim:]
        z = reparametrize(mu, logvar)
        x_recon = self.decoder(z)
        if not train:
          return x_recon, z
          
        return x_recon, mu, logvar


In [ ]:
def kaiming_init(m):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        init.kaiming_normal_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0)
    elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
        m.weight.data.fill_(1)
        if m.bias is not None:
            m.bias.data.fill_(0)


def normal_init(m, mean, std):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        m.weight.data.normal_(mean, std)
        if m.bias.data is not None:
            m.bias.data.zero_()
    elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
        m.weight.data.fill_(1)
        if m.bias.data is not None:
            m.bias.data.zero_()


In [ ]:
model = BetaVAE_H()
print(model)

In [ ]:
def train(model, optimiser, dataloader, beta):
  model.train()
  # self.C_max = Variable(cuda(torch.FloatTensor([self.C_max]), self.use_cuda))
  rl = 0
  bl = 0
  kl = 0
  trl = []
  tbl = []
  tkl = []
  for i, x in enumerate(dataloader):
    x = x.cuda()
    x_recon, mu, logvar = model(x)
    recon_loss = reconstruction_loss(x, x_recon, 'gaussian')
    total_kld, dim_wise_kld, mean_kld = kl_divergence(mu, logvar)

    beta_vae_loss = recon_loss + beta*total_kld
    
    optimiser.zero_grad()
    beta_vae_loss.backward()
    optimiser.step()
    
    rl+=recon_loss.item()
    kl+=total_kld.item()
    bl+=beta_vae_loss.item()

    if i % 20 == 0:
        print('[{}] recon_loss:{:.3f} total_kld:{:.3f} mean_kld:{:.3f} beta_vae_loss:{:.3f}'.format(
            i, recon_loss.item(), total_kld.item(), mean_kld.item(), beta_vae_loss.item()))

        # var = logvar.exp().mean(0).data
        # var_str = ''
        # for j, var_j in enumerate(var):
        #     var_str += 'var{}:{:.4f} '.format(j+1, var_j)
        # print(var_str)

        trl.append(recon_loss.item())
        tkl.append(total_kld.item())
        tbl.append(beta_vae_loss.item())

  return rl/len(dataloader), bl/len(dataloader), kl/len(dataloader), trl, tbl, tkl

In [ ]:
model = BetaVAE_H()
model.cuda()

optimiser = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=100, gamma=0.5)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode="min", factor=0.3, patience=1, verbose=True)
epochs = 100
model_no = 2
beta = 10

TRL, TBL, TKL, RL, BL, KL = [], [], [], [], [], []

for i in range(epochs):
    if path.exists('model_'+ str(model_no) + '_' + str(i)):
        print("Load model for epoch", i)
        checkpoint = torch.load('model_' + str(model_no) + '_' + str(i))
        model.load_state_dict(checkpoint['model_state_dict'])
        # optimiser.load_state_dict(checkpoint['optimiser_state_dict'])
        # scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        i = checkpoint['epoch']
        TRL = checkpoint['trl']
        TBL = checkpoint['tbl']
        TKL = checkpoint['tkl']
        RL = checkpoint['rl']
        BL = checkpoint['bl']
        KL = checkpoint['kl']
        
        continue

    # break    
    print(scheduler.get_last_lr())
    
    print("Training epoch", i)
    start = time.time()
    rl, bl, kl, trl, tbl, tkl = train(model, optimiser, loader, beta)
    print("RL:", rl, "BL:", bl, "KL:", kl)
    print("Train time:", time.time()-start)
    
    TRL += trl
    TBL += tbl
    TKL += tkl
    RL.append(rl)
    BL.append(bl)
    KL.append(kl)
    
    # scheduler.step()
    
    torch.save({
                'epoch': i,
                'model_state_dict': model.state_dict(),
                'optimiser_state_dict': optimiser.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'trl': TRL,
                'tbl': TBL,
                'tkl': TKL,
                'rl': RL,
                'bl': BL,
                'kl': KL,
                }, 'model_' + str(model_no) + '_' + str(i))


In [ ]:

# Model 0 - Beta = 250, gaussian
# Model 1 - Beta = 4, gaussian
# Model 2 - Beta = 10, gaussian
# Model 4 - Beta = 1, gaussian
# Model 3 - Beta = 250, bernoulli